In [ ]:
import mujoco
import mujoco.viewer
import matplotlib.pyplot as plt
import mediapy as media
import numpy as np
import sympy as sp
import os
import itertools
import time
from scipy.spatial.transform import Rotation
import plotly.graph_objects as go
import mujoco as mj

### 0. Initialization

In [ ]:
all_pts = np.empty((0,3), dtype=float)
x_min, x_max  = 0.09, 1   # It will only plot coordinates under this magnitude in x,y,z
y_min, y_max  = 0.05, 1
z_min ,z_max = 0.06, 3
noise_std = 0.01 # Noise std for depth sensing

# Camera parameters
camera_name = 'ee_camera'
fovy_deg = 90

height = 100
width = 120

frames1 = []
frames2 = []

with open('robotic_arm_xml.xml', 'r') as f:
    xml_string = f.read()

# Functions: 

def run_simulation(joint_positions_angles, camera_name, fovy_deg):

    xml_filled = xml_string.format(camera=camera_name, fovy=fovy_deg)

    model = mujoco.MjModel.from_xml_string(xml_filled)
    data = mujoco.MjData(model)

    for i, angle in enumerate(joint_positions_angles):
        # Slider joint
        if i == 0:
            data.qpos[i] = angle
        # Revolute joints
        else:
            data.qpos[i] = np.deg2rad(angle)

    # print("joint_positions_angles: ",joint_positions_angles)
    mujoco.mj_forward(model, data)

    # Big image print
    with mujoco.Renderer(model, 480, 640) as renderer2:
        renderer2.update_scene(data)
        rgb2 = renderer2.render()
        # media.show_image(renderer2.render())
        frames1.append(renderer2.render())

    # Render camera RGB
    with mujoco.Renderer(model, height, width) as renderer:
        renderer.update_scene(data, camera_name)
        rgb = renderer.render()
        # media.show_image(rgb)
        frames2.append(renderer.render())

        # Render Depth
        renderer.enable_depth_rendering()
        renderer.update_scene(data, camera_name)
        depth_real = renderer.render()
        
        # Adding distance-dependent Gaussian noisee
        noise_std = 0.001 + 0.01 * depth_real
        noise = np.random.normal(0, noise_std)
        depth_noisy = depth_real + noise
        depth_noisy = np.clip(depth_noisy, 0.01, None)

    # # Live viewer
    # with mujoco.viewer.launch_passive(model, data) as viewer:
    #     while viewer.is_running():
    #         step_start = data.time
    #         mujoco.mj_step(model, data)
    #         viewer.sync()

    return depth_noisy, depth_real, renderer, data, model, rgb2, rgb

def find_points_camera(depth, fovy):
  # Suppose depth is shape (H, W)
  H, W = depth.shape
  fovy = np.deg2rad(fovy) 
  fy = H / (2 * np.tan(fovy / 2))
  fx = fy
  cx, cy = W / 2, H / 2    # assuming center principal point

  u, v = np.meshgrid(np.arange(W), np.arange(H))

  Z = depth
  X = (u - cx) * Z / fx
  Y = (v - cy) * Z / fy
  points_camera = np.stack((X, Y, Z), axis=-1)  # shape (H, W, 3)

  return points_camera

def find_world_coords(renderer, data, model, points_camera, camera_name=None):
    """Returns the 3x4 camera matrix (image @ focal @ rotation @ translation).
    If camera_name is None → use the default camera."""
    
    global all_pts

    if camera_name:
        renderer.update_scene(data, camera_name)
        cam_name = camera_name
    else:
        renderer.update_scene(data)
        cam_name = None

    # --- camera id & fovy ---
    if cam_name is not None:
        cam_id = mj.mj_name2id(model, mj.mjtObj.mjOBJ_CAMERA, cam_name)
        fovy = float(model.cam_fovy[cam_id])

    else:
        fovy = float(model.vis.global_.fovy)
        translation = np.zeros(3)


    #print(f"Camera '{cam_name if cam_name else 'default'}' FOVY: {fovy}°")

    translation = np.array(data.cam_xpos[cam_id], dtype=float)  # (3,) world position
    rotation = np.array(data.cam_xmat[cam_id], dtype=float).reshape(3, 3)  # (3,3) world orientation
    # display("translation:", sp.Matrix(translation))
    # display("rotation:", sp.Matrix(rotation))
    
    R_x_180 = np.array([
            [1,  0,  0],
            [0, -1,  0],
            [0,  0, -1]
        ])

    pts = points_camera.reshape(-1, 3)
    R_combined = rotation @ R_x_180          # camera->world * corrective

    pts_world = pts @ R_combined.T + translation.reshape(1, 3)   # (N,3)

    mask = (np.abs(pts_world[:, 0]) < x_max) & (np.abs(pts_world[:, 0]) > x_min) & (np.abs(pts_world[:, 1]) < y_max) & (np.abs(pts_world[:, 1]) > y_min) & (np.abs(pts_world[:, 2] )< z_max) & (np.abs(pts_world[:, 2]) > z_min)
    pts_filtered = pts_world[mask]

    # if no new points, all_pts stays same
    if pts_filtered.size > 0:
        all_pts = np.vstack((all_pts, pts_filtered))
        all_pts = np.unique(all_pts, axis=0)
        # print("New count of all_pts: ", all_pts.shape)

    return pts_filtered, all_pts


### 1. Camera setup & scan

In [ ]:
# Running the main code:

# interpolation
joint1_range = np.linspace(-0.5, 0.5, 40) #slider only

# Base joint positions
base_angles = [-0.5, 0, -40, 40, -60, -40, -30]

frames1 = []
frames2 = []

# Loop
for i, j1 in enumerate(joint1_range):
    joint_positions_angles = base_angles.copy()
    joint_positions_angles[0] = j1

    depth_noisy, depth_real, renderer, data, model, rgb2, rgb = run_simulation(joint_positions_angles, camera_name, fovy_deg)
    # if i==4:
    #     media.show_image(rgb2)
    #     media.show_image(rgb)
        
    points_camera = find_points_camera(depth_noisy, fovy_deg)
    pts_filtered, all_pts_new = find_world_coords(renderer, data, model, points_camera, camera_name)

media.show_video(frames1, fps=10)
media.show_video(frames2, fps=10)

In [ ]:
# Running the main code:

# interpolation
joint1_range = np.linspace(-1, 1, 40)
joint5_range = np.linspace(-180, -30, 40)

# Base joint positions
base_angles = [-0.5, 90, -60, 40, -60, -140, -30]

frames1 = []
frames2 = []

# Loop
for i, (j1, j5) in enumerate(zip(joint1_range, joint5_range)):
    joint_positions_angles = base_angles.copy()
    joint_positions_angles[0] = j1
    joint_positions_angles[5] = j5

    depth_noisy, depth_real, renderer, data, model, rgb2, rgb = run_simulation(joint_positions_angles, camera_name, fovy_deg)
    # if i==0:
    #     media.show_image(rgb2)
    #     media.show_image(rgb)
        
    points_camera = find_points_camera(depth_noisy, fovy_deg)
    pts_filtered, all_pts_new = find_world_coords(renderer, data, model, points_camera, camera_name)

media.show_video(frames1, fps=10)
media.show_video(frames2, fps=10)

### 2. Plotting the points

In [ ]:
# Plot all points
arrow_len = 0.1

fig = go.Figure([
    go.Scatter3d(
        x=all_pts[:,0], y=all_pts[:,1], z=all_pts[:,2],
        mode='markers',
        marker=dict(size=1, color=all_pts[:,2], colorscale='Viridis', opacity=0.8)
    ),
    # X-axis (red)
    go.Scatter3d(x=[0, arrow_len], y=[0, 0], z=[0, 0], mode='lines+text',
                line=dict(color='red', width=6), name='X',
                text=['', 'X'], textposition='top center', textfont=dict(size=14, color='red')),
    # Y-axis (green)
    go.Scatter3d(x=[0, 0], y=[0, arrow_len], z=[0, 0], mode='lines+text',
                line=dict(color='green', width=6), name='Y',
                text=['', 'Y'], textposition='top center', textfont=dict(size=14, color='green')),
    # Z-axis (blue)
    go.Scatter3d(x=[0, 0], y=[0, 0], z=[0, arrow_len], mode='lines+text',
                line=dict(color='blue', width=6), name='Z',
                text=['', 'Z'], textposition='top center', textfont=dict(size=14, color='blue')),
])

fig.update_layout(
    width=1000,
    height=600,
    margin=dict(l=0, r=0, t=0, b=0),
    scene=dict(
        xaxis_title='X (m)',
        yaxis_title='Y (m)',
        zaxis_title='Z (m)',
        aspectmode='data'
    )
)

fig.show()

### 3. Surface plotting:

In [ ]:
import open3d as o3d
import numpy as np
import plotly.graph_objects as go

# Numpy array to Open3D point cloud
pcd = o3d.geometry.PointCloud()
pcd.points = o3d.utility.Vector3dVector(all_pts)

# Estimate normals
pcd.estimate_normals(
    search_param=o3d.geometry.KDTreeSearchParamHybrid(radius=0.1, max_nn=30)
)

# Orient normals consistently
pcd.orient_normals_consistent_tangent_plane(k=15)

# Poisson surface reconstruction (creates smooth surface)
mesh, densities = o3d.geometry.TriangleMesh.create_from_point_cloud_poisson(
    pcd, depth=9
)

# Remove low-density vertices (cleanup)
vertices_to_remove = densities < np.quantile(densities, 0.03)
mesh.remove_vertices_by_mask(vertices_to_remove)

# Smooth the mesh more to reduce noise
mesh = mesh.filter_smooth_simple(number_of_iterations=13)
mesh.compute_vertex_normals()

# Extract vertices and triangles for Plotly
vertices = np.asarray(mesh.vertices)
triangles = np.asarray(mesh.triangles)

# Create the surface plot with coordinate axes
arrow_len = 0.1

fig = go.Figure([
    # Surface mesh
    go.Mesh3d(
        x=vertices[:, 0],
        y=vertices[:, 1],
        z=vertices[:, 2],
        i=triangles[:, 0],
        j=triangles[:, 1],
        k=triangles[:, 2],
        intensity=vertices[:, 2],
        colorscale='Viridis',
        opacity=1,
        name='Surface',
        showscale=True,
        flatshading=False
    ),
    # X-axis (red)
    go.Scatter3d(x=[0, arrow_len], y=[0, 0], z=[0, 0], mode='lines+text',
                line=dict(color='red', width=6), name='X',
                text=['', 'X'], textposition='top center', textfont=dict(size=14, color='red')),
    # Y-axis (green)
    go.Scatter3d(x=[0, 0], y=[0, arrow_len], z=[0, 0], mode='lines+text',
                line=dict(color='green', width=6), name='Y',
                text=['', 'Y'], textposition='top center', textfont=dict(size=14, color='green')),
    # Z-axis (blue)
    go.Scatter3d(x=[0, 0], y=[0, 0], z=[0, arrow_len], mode='lines+text',
                line=dict(color='blue', width=6), name='Z',
                text=['', 'Z'], textposition='top center', textfont=dict(size=14, color='blue')),
])

fig.update_layout(
    width=1000,
    height=600,
    margin=dict(l=0, r=0, t=0, b=0),
    scene=dict(
        xaxis_title='X (m)',
        yaxis_title='Y (m)',
        zaxis_title='Z (m)',
        aspectmode='data'
    )
)

fig.show()

In [ ]:
import open3d as o3d
import numpy as np
import plotly.graph_objects as go

# Convert your numpy array to Open3D point cloud
pcd = o3d.geometry.PointCloud()
pcd.points = o3d.utility.Vector3dVector(all_pts)

# Estimate normals with good parameters
pcd.estimate_normals(
    search_param=o3d.geometry.KDTreeSearchParamHybrid(radius=0.1, max_nn=30)
)

# Orient normals consistently
pcd.orient_normals_consistent_tangent_plane(k=15)

# Poisson surface reconstruction 
mesh, densities = o3d.geometry.TriangleMesh.create_from_point_cloud_poisson(
    pcd, depth=9, width=0, scale=1.1, linear_fit=False
)

# Remove low-density vertices
vertices_to_remove = densities < np.quantile(densities, 0.03)
mesh.remove_vertices_by_mask(vertices_to_remove)

# Apply multiple smoothing passes
mesh = mesh.filter_smooth_simple(number_of_iterations=20)

# Compute vertex normals
mesh.compute_vertex_normals()

# Extract vertices and triangles for Plotly
vertices = np.asarray(mesh.vertices)
triangles = np.asarray(mesh.triangles)
normals = np.asarray(mesh.vertex_normals)

# Sample vertices to show normals (showing all would be too cluttered)
sample_indices = np.random.choice(len(vertices), size=min(500, len(vertices)), replace=False)
sampled_vertices = vertices[sample_indices]
sampled_normals = normals[sample_indices]

# Create normal vectors for visualization
normal_scale = 0.02  # Length of normal arrows
normal_lines = []
for i in range(len(sampled_vertices)):
    start = sampled_vertices[i]
    end = start + sampled_normals[i] * normal_scale
    normal_lines.append(go.Scatter3d(
        x=[start[0], end[0], None],
        y=[start[1], end[1], None],
        z=[start[2], end[2], None],
        mode='lines',
        line=dict(color='cyan', width=2),
        showlegend=False,
        hoverinfo='skip'
    ))

# Create the surface plot with coordinate axes
arrow_len = 0.1

fig = go.Figure([
    # Surface mesh
    go.Mesh3d(
        x=vertices[:, 0],
        y=vertices[:, 1],
        z=vertices[:, 2],
        i=triangles[:, 0],
        j=triangles[:, 1],
        k=triangles[:, 2],
        intensity=vertices[:, 2],  # Color by Z-coordinate
        colorscale='Viridis',
        opacity=0.9,
        name='Surface',
        showscale=True,
        flatshading=False,
        lighting=dict(ambient=0.6, diffuse=0.9, specular=0.1, roughness=0.5, fresnel=0.2)
    ),
    *normal_lines,  # Add all normal lines
    # X-axis (red)
    go.Scatter3d(x=[0, arrow_len], y=[0, 0], z=[0, 0], mode='lines+text',
                line=dict(color='red', width=6), name='X',
                text=['', 'X'], textposition='top center', textfont=dict(size=14, color='red')),
    # Y-axis (green)
    go.Scatter3d(x=[0, 0], y=[0, arrow_len], z=[0, 0], mode='lines+text',
                line=dict(color='green', width=6), name='Y',
                text=['', 'Y'], textposition='top center', textfont=dict(size=14, color='green')),
    # Z-axis (blue)
    go.Scatter3d(x=[0, 0], y=[0, 0], z=[0, arrow_len], mode='lines+text',
                line=dict(color='blue', width=6), name='Z',
                text=['', 'Z'], textposition='top center', textfont=dict(size=14, color='blue')),
])

fig.update_layout(
    width=1000,
    height=600,
    margin=dict(l=0, r=0, t=0, b=0),
    scene=dict(
        xaxis_title='X (m)',
        yaxis_title='Y (m)',
        zaxis_title='Z (m)',
        aspectmode='data'
    )
)

fig.show()